In [19]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [20]:
# 2

file_path = "gmo_analysis_data.xlsx"
tr = pd.read_excel(file_path, sheet_name="total returns")
rf = pd.read_excel(file_path, sheet_name="risk-free rate")
signal = pd.read_excel(file_path, sheet_name="signals")
tr["date"] = pd.to_datetime(tr["date"])
rf["date"] = pd.to_datetime(rf["date"])
signal["date"] = pd.to_datetime(signal["date"])
tr.set_index("date", inplace=True)
rf.set_index("date", inplace=True)
signal.set_index("date", inplace=True) 

er = tr - rf.values
er.head()

,SPY,GMWAX,GMGEX
date,,,
1996-12-31,-0.075002,-0.073804,-0.064710
1997-01-31,0.010316,-0.036735,-0.017022
1997-02-28,-0.042635,-0.029935,-0.039467
1997-03-31,-0.098941,-0.068372,-0.069661
1997-04-30,0.012038,-0.059061,-0.052330


In [21]:
FREQ = 12

def performance_stats(ret):
    """Compute mean, vol, Sharpe (annualized)."""
    mean_ann = ret.mean() * FREQ
    vol_ann = ret.std() * np.sqrt(FREQ)
    sharpe = mean_ann / vol_ann if vol_ann != 0 else np.nan
    return mean_ann, vol_ann, sharpe

def max_drawdown(total_ret):
    wealth = (1 + total_ret).cumprod()
    peak = wealth.cummax()
    dd = (wealth - peak) / peak
    return dd.min()

def tail_risk(ret, total_ret):
    min_ret = ret.min()
    var_5 = ret.quantile(0.05)
    mdd = max_drawdown(total_ret)
    return min_ret, var_5, mdd

def regression_stats(y, x):
    X = sm.add_constant(x)
    model = sm.OLS(y, X).fit()
    alpha_daily = model.params["const"]
    beta = model.params[x.name]
    r2 = model.rsquared
    alpha_ann = alpha_daily * FREQ
    return alpha_ann, beta, r2

In [22]:
# 2.1

samples = {
    "inception_to_2011": er.loc[: "2011-12-31"],
    "2012_to_present":   er.loc["2012-01-01": ],
    "inception_to_present": er
}

print("===== 2.1 PERFORMANCE (GMWAX) =====")
for name, sub in samples.items():
    m, v, s = performance_stats(sub["GMWAX"])
    print(f"\n--- {name} ---")
    print(f"Mean (ann):   {m:.6f}")
    print(f"Vol (ann):    {v:.6f}")
    print(f"Sharpe:       {s:.6f}")

===== 2.1 PERFORMANCE (GMWAX) =====

--- inception_to_2011 ---
Mean (ann):   -0.265291
Vol (ann):    0.131867
Sharpe:       -2.011809

--- 2012_to_present ---
Mean (ann):   -0.123303
Vol (ann):    0.109588
Sharpe:       -1.125154

--- inception_to_present ---
Mean (ann):   -0.197366
Vol (ann):    0.123263
Sharpe:       -1.601179


<span style="color: blue;">

ANS:

The mean became less negative in sub_period 2, while volatility also become less, the Sharpe ratio improve in the sub_period 2, from -2.0 to -1.1.

In [23]:
#2.2

print("\n===== 2.2 TAIL RISK (GMWAX) =====")
for name, sub in samples.items():
    min_ret, var5, mdd = tail_risk(
        sub["GMWAX"],            # excess return
        tr.loc[sub.index]["GMWAX"]    # total return
    )
    print(f"\n--- {name} ---")
    print(f"Min return:   {min_ret:.6f}")
    print(f"VaR-5%:       {var5:.6f}")
    print(f"Max drawdown: {mdd:.6f}")


===== 2.2 TAIL RISK (GMWAX) =====

--- inception_to_2011 ---
Min return:   -0.193379
VaR-5%:       -0.083767
Max drawdown: -0.293614

--- 2012_to_present ---
Min return:   -0.115577
VaR-5%:       -0.071114
Max drawdown: -0.216795

--- inception_to_present ---
Min return:   -0.193379
VaR-5%:       -0.077006
Max drawdown: -0.293614


<span style="color: blue;">

ANS:

(a) The tail-risk seems high, as Var-5% of monthly data is 7-8% through the sample, and the minimum return is 19% in all the sample, and maximum drawdown is 29.4% in all the sample.

(b) Yes, the second sub-sample seems less ricky in term of tail risk. But it is still quite risky as the Var-5% is 7.1%, maximum drawdown is 22%, and minimum return per month is 11.6%.

In [24]:
# 2.3

print("\n===== 2.3 MARKET EXPOSURE (GMWAX) =====")
for name, sub in samples.items():
    alpha, beta, r2 = regression_stats(
        sub["GMWAX"],     # y
        sub["SPY"]  # x = SPY excess
    )
    print(f"\n--- {name} ---")
    print(f"Alpha (ann):  {alpha:.6f}")
    print(f"Beta:         {beta:.6f}")
    print(f"R²:           {r2:.6f}")



===== 2.3 MARKET EXPOSURE (GMWAX) =====

--- inception_to_2011 ---
Alpha (ann):  -0.094369
Beta:         0.619535
R²:           0.687802

--- 2012_to_present ---
Alpha (ann):  -0.099708
Beta:         0.629431
R²:           0.769300

--- inception_to_present ---
Alpha (ann):  -0.096637
Beta:         0.622398
R²:           0.727059


<span style="color: blue;">

ANS:

GMWAX has around 0.6 Beta, I would not consider it low-beta strategy, and this didn't change across subsamples.

GMWAX does not provide alpha, it has negative alpha across both subsamples.

In [25]:
# 2.4

def analyze_one_fund(fund_name):
    print(f"\n========== RESULTS FOR {fund_name} ==========")

    # 2.1 Performance
    print("\n--- 2.1 PERFORMANCE ---")
    for name, sub in samples.items():
        m, v, s = performance_stats(sub[fund_name])
        print(f"\n{name}")
        print(f"Mean (ann):   {m:.6f}")
        print(f"Vol (ann):    {v:.6f}")
        print(f"Sharpe:       {s:.6f}")

    # 2.2 Tail Risk
    print("\n--- 2.2 TAIL RISK ---")
    for name, sub in samples.items():
        min_ret, var5, mdd = tail_risk(
            sub[fund_name],
            tr.loc[sub.index][fund_name]
        )
        print(f"\n{name}")
        print(f"Min return:   {min_ret:.6f}")
        print(f"VaR-5%:       {var5:.6f}")
        print(f"Max drawdown: {mdd:.6f}")

    # 2.3 Market Exposure Regression
    print("\n--- 2.3 REGRESSION ---")
    for name, sub in samples.items():
        alpha, beta, r2 = regression_stats(
            sub[fund_name],
            er.loc[sub.index]["SPY"]
        )
        print(f"\n{name}")
        print(f"Alpha (ann):  {alpha:.6f}")
        print(f"Beta:         {beta:.6f}")
        print(f"R²:           {r2:.6f}")

# Run comparison
analyze_one_fund("GMWAX")
analyze_one_fund("GMGEX")


========== RESULTS FOR GMWAX ==========

--- 2.1 PERFORMANCE ---

inception_to_2011
Mean (ann):   -0.265291
Vol (ann):    0.131867
Sharpe:       -2.011809

2012_to_present
Mean (ann):   -0.123303
Vol (ann):    0.109588
Sharpe:       -1.125154

inception_to_present
Mean (ann):   -0.197366
Vol (ann):    0.123263
Sharpe:       -1.601179

--- 2.2 TAIL RISK ---

inception_to_2011
Min return:   -0.193379
VaR-5%:       -0.083767
Max drawdown: -0.293614

2012_to_present
Min return:   -0.115577
VaR-5%:       -0.071114
Max drawdown: -0.216795

inception_to_present
Min return:   -0.193379
VaR-5%:       -0.077006
Max drawdown: -0.293614

--- 2.3 REGRESSION ---

inception_to_2011
Alpha (ann):  -0.094369
Beta:         0.619535
R²:           0.687802

2012_to_present
Alpha (ann):  -0.099708
Beta:         0.629431
R²:           0.769300

inception_to_present
Alpha (ann):  -0.096637
Beta:         0.622398
R²:           0.727059

========== RESULTS FOR GMGEX ==========

--- 2.1 PERFORMANCE ---

incepti

<span style="color: blue;">

ANS:

Performance Comparison

GMGEX underperforms GMWAX consistently, showing lower annualized returns and significantly higher volatility.

Both funds have negative Sharpe ratios, but GMGEX’s Sharpe is worse, especially in the post-2012 period.

Tail-Risk Comparison

GMGEX exhibits substantially higher tail risk, including a maximum single-day loss of –66% and maximum drawdowns deeper than –70%.

GMWAX’s worst outcomes are far milder, with drawdowns around –30%.

Market Exposure

GMWAX has a stable low beta (~0.62), consistent with a lower-risk profile.

GMGEX has a higher beta (~0.80) and less stable market exposure (R² drops to ~0.28 after 2012).

Both funds deliver negative alpha, but GMGEX’s alpha deteriorates more severely.

Overall Conclusion

GMWAX is a lower-risk, lower-beta, but low-return strategy.
GMGEX carries much higher risk, larger drawdowns, unstable behavior, and even worse performance.


In [26]:
# 3

df = tr.join(rf, how="inner", rsuffix="_rf").join(signal, how="inner")
df["er"] = df["SPY"] - df["TBill 3M"]

df["dp_lag"] = df["SPX D/P"].shift(1)
df["ep_lag"] = df["SPX E/P"].shift(1)
df["y10_lag"] = df["T-Note 10YR"].shift(1)

df = df.dropna()

def regress(y, X):
    X = sm.add_constant(X)
    return sm.OLS(y, X).fit()

def compute_max_dd(series):
    cum = (1 + series).cumprod()
    peak = cum.cummax()
    dd = (cum - peak) / peak
    return dd.min()

def information_ratio(alpha, residuals):
    return alpha / residuals.std()

In [27]:
# 3.1

y = df["SPY"]

model_dp = regress(y, df[["dp_lag"]])
model_ep = regress(y, df[["ep_lag"]])
model_three = regress(y, df[["dp_lag", "ep_lag", "y10_lag"]])

print("===== REGRESSIONS =====")
print("(1) dp-lag")
print("R²:", round(model_dp.rsquared, 6))
print(model_dp.params.round(6))

print("\n(2) ep-lag")
print("R²:", round(model_ep.rsquared, 6))
print(model_ep.params.round(6))

print("\n(3) dp + ep + y10")
print("R²:", round(model_three.rsquared, 6))
print(model_three.params.round(6))

===== REGRESSIONS =====
(1) dp-lag
R²: 0.007263
const    -0.007793
dp_lag    0.928617
dtype: float64

(2) ep-lag
R²: 0.004827
const    -0.004074
ep_lag    0.240449
dtype: float64

(3) dp + ep + y10
R²: 0.008638
const     -0.002655
dp_lag     0.445540
ep_lag     0.142754
y10_lag   -0.117296
dtype: float64


In [28]:
# 3.2

def build_strategy(model, Xlag):
    Xlag = sm.add_constant(Xlag)
    forecast = model.predict(Xlag)      # E_t[er_{t+1}]
    weight = forecast * 100                  # w_t = forecast
    strategy_ret = weight * df["SPY"]    # realized return
    return strategy_ret, forecast

ret_dp, fc_dp = build_strategy(model_dp, df[["dp_lag"]])
ret_ep, fc_ep = build_strategy(model_ep, df[["ep_lag"]])
ret_three, fc_three = build_strategy(model_three, df[["dp_lag", "ep_lag", "y10_lag"]])

def perf_stats(strategy_ret):
    mean = strategy_ret.mean()
    vol = strategy_ret.std()
    sharpe = mean / vol

    maxdd = compute_max_dd(strategy_ret)

    X = sm.add_constant(df["SPY"])
    mkt_model = sm.OLS(strategy_ret, X).fit()
    alpha = mkt_model.params["const"]
    beta = mkt_model.params["SPY"]
    ir = information_ratio(alpha, mkt_model.resid)

    return {
        "mean": round(mean,6),
        "vol": round(vol,6),
        "sharpe": round(sharpe,6),
        "maxdd": round(maxdd,6),
        "alpha": round(alpha,6),
        "beta": round(beta,6),
        "IR": round(ir,6),
    }

stats_dp = perf_stats(ret_dp)
stats_ep = perf_stats(ret_ep)
stats_three = perf_stats(ret_three)

print("\n===== STRATEGY PERFORMANCE =====")
print("DP:", stats_dp)
print("EP:", stats_ep)
print("Three:", stats_three)


===== STRATEGY PERFORMANCE =====
DP: {'mean': np.float64(0.00932), 'vol': np.float64(0.048565), 'sharpe': np.float64(0.191919), 'maxdd': np.float64(-0.698615), 'alpha': np.float64(0.000711), 'beta': np.float64(0.96872), 'IR': np.float64(0.031287)}
EP: {'mean': np.float64(0.008844), 'vol': np.float64(0.044907), 'sharpe': np.float64(0.196935), 'maxdd': np.float64(-0.616841), 'alpha': np.float64(0.000416), 'beta': np.float64(0.948248), 'IR': np.float64(0.026243)}
Three: {'mean': np.float64(0.00959), 'vol': np.float64(0.048226), 'sharpe': np.float64(0.198846), 'maxdd': np.float64(-0.666069), 'alpha': np.float64(0.001045), 'beta': np.float64(0.96144), 'IR': np.float64(0.046211)}


In [29]:
# 3.3

VaR = {
    "dp": round(ret_dp.quantile(0.05),6),
    "ep": round(ret_ep.quantile(0.05),6),
    "three": round(ret_three.quantile(0.05),6),
    "market": round(df["SPY"].quantile(0.05),6),
}

print("\n===== VAR(5%) =====")
print(VaR)


df_sub = df.loc["2000-01-01":"2011-12-31"]

def underperf(ret):
    return round((ret.loc[df_sub.index] - df_sub["TBill 3M"]).mean(),6)

print("\n===== UNDERPERFORMANCE VS RF 2000–2011 =====")
print("dp:", underperf(ret_dp))
print("ep:", underperf(ret_ep))
print("three:", underperf(ret_three))

print("\n===== NEGATIVE RISK PREMIUM COUNTS =====")
print("dp:", (fc_dp < 0).sum())
print("ep:", (fc_ep < 0).sum())
print("three:", (fc_three < 0).sum())


===== VAR(5%) =====
{'dp': np.float64(-0.061545), 'ep': np.float64(-0.064144), 'three': np.float64(-0.065216), 'market': np.float64(-0.074428)}

===== UNDERPERFORMANCE VS RF 2000–2011 =====
dp: -0.018652
ep: -0.019573
three: -0.018332

===== NEGATIVE RISK PREMIUM COUNTS =====
dp: 0
ep: 0
three: 1


<span style="color: blue;">

ANS:

Do you believe the dynamic strategy takes on extra risk?
I don't believe so, and if we look at the var-5%, the dynamic strategy perform better than SPY. 
However, the three factor dynamic strategy shows a 0.1% alpha (for monthly data). This may mean it take extra risk base on markey efficiency theory.